## Libraries & Configuration

In [9]:
# Libraries
import numpy as np
import math
from scipy.linalg import block_diag
from matplotlib import pyplot as plt
import copy
from mpl_toolkits import mplot3d
import numpy as np
import pandas as pd 

# Configurations
%matplotlib qt
plt.rcParams['figure.figsize']=[15,15]

## Function Declarations

In [2]:
# Transformation rules for adjacency matrix, Z

def transform_Z(symplectic, Z):
    A = symplectic[:int(np.shape(symplectic)[0]/2),:int(np.shape(symplectic)[0]/2)]
    B = symplectic[int(np.shape(symplectic)[0]/2):,:int(np.shape(symplectic)[0]/2)]
    C = symplectic[:int(np.shape(symplectic)[0]/2),int(np.shape(symplectic)[0]/2):]
    D = symplectic[int(np.shape(symplectic)[0]/2):,int(np.shape(symplectic)[0]/2):]

    return (C + D@Z)@np.linalg.inv(A + B@Z)

# Mode transformation from H-graph into Approximate CV Cluster state

def identify_bipartition(Z):
    modes = list(np.arange(int(np.shape(Z)[0])))
    for mode in modes:
        test_modes = copy.deepcopy(modes)
        test_modes.remove(mode)
        for test_mode in test_modes:
            if abs(Z[mode][test_mode]) > 0.1 or abs(Z[test_mode][mode]) > 0.1:
                modes.remove(test_mode)
    return modes

def generate_S_h2c(modes):
    S_h2c = np.identity(num_modes*4)
    for mode in modes:
        S_h2c[mode][mode] = 0
        S_h2c[mode][mode+2*num_modes] = -1
        S_h2c[mode+2*num_modes][mode] = 1
        S_h2c[mode+2*num_modes][mode+2*num_modes] = 0

    return S_h2c

# Symplectic matrix for EPR state generation from two initially squeezed modes

def epr_symplectic():

    #Delay on B mode

    S_phase_delay = np.zeros((num_modes*4, num_modes*4))

    for i in range(num_modes):
        S_phase_delay[i*2,i*2] = 1
        S_phase_delay[(num_modes+i)*2,(num_modes+i)*2] = 1
        S_phase_delay[i*2+1,(num_modes+i)*2+1] = -1
        S_phase_delay[(num_modes+i)*2+1,i*2+1] = 1

    #Beamsplitter for EPR state generation

    S_bs0 = np.array([])

    for i in range(num_modes):
        bs_block = np.array([[1/np.sqrt(2),-1/np.sqrt(2)],[1/np.sqrt(2),1/np.sqrt(2)]])
        if i == 0:
            S_bs0 = np.block([
                [bs_block, np.zeros((2,2))],
                [np.zeros((2,2)), bs_block]
            ])
        else:
            size = np.shape(S_bs0)
            S_bs0 = np.block([
                [S_bs0[:int(size[0]/2),:int(size[0]/2)],np.zeros((int(size[1]/2),2)), S_bs0[int(size[0]/2):,:int(size[0]/2)],np.zeros((int(size[1]/2),2))],
                [np.zeros((2,int(size[0]/2))), bs_block, np.zeros((2,int(size[0]/2))), np.zeros((2,2))],
                [S_bs0[int(size[0]/2):,:int(size[0]/2)], np.zeros((int(size[1]/2),2)), S_bs0[int(size[0]/2):,int(size[0]/2):], np.zeros((int(size[1]/2),2))],
                [np.zeros((2,int(size[0]/2))), np.zeros((2,2)), np.zeros((2,int(size[0]/2))), bs_block]
            ])

    return S_bs0@S_phase_delay

# Symplectic matrix for cluster state generation from an EPR state

def cluster_symplectic(bs_1=0, bs_2=0, bs_3=0):

    #Beamsplitter for 1D cluster generation

    if bs_1 == 0:

        S_bs1 = np.diag(np.ones(num_modes*4))

    else:

        bs1_block = np.diag(np.ones(num_modes*2))

        for i in range(num_modes):
            if 2*i+1+(2*bs_1-1) < num_modes*2:
                bs1_block[2*i+1,2*i+1] = 1/np.sqrt(2)
                bs1_block[2*i+1,2*i+1+(2*bs_1-1)] = -1/np.sqrt(2)
                bs1_block[2*i+1+(2*bs_1-1),2*i+1] = 1/np.sqrt(2)
                bs1_block[2*i+1+(2*bs_1-1),2*i+1+(2*bs_1-1)] = 1/np.sqrt(2)

        S_bs1 = np.block([[bs1_block, np.zeros((num_modes*2, num_modes*2))],[np.zeros((num_modes*2, num_modes*2)), bs1_block]]) 

    #Beamsplitter for 2D cluster generation

    if bs_2 == 0:

        S_bs2 = np.diag(np.ones(num_modes*4))

    else:

        bs2_block = np.diag(np.ones(num_modes*2))

        for i in range(num_modes):
            if 2*i+1+(2*bs_2-1) < num_modes*2:
                bs2_block[2*i+1,2*i+1] = 1/np.sqrt(2)
                bs2_block[2*i+1,2*i+1+(2*bs_2-1)] = -1/np.sqrt(2)
                bs2_block[2*i+1+(2*bs_2-1),2*i+1] = 1/np.sqrt(2)
                bs2_block[2*i+1+(2*bs_2-1),2*i+1+(2*bs_2-1)] = 1/np.sqrt(2)

        S_bs2 = np.block([[bs2_block, np.zeros((num_modes*2, num_modes*2))],[np.zeros((num_modes*2, num_modes*2)), bs2_block]])  

    #Beamsplitter for 3D cluster generation

    if bs_3 == 0:

        S_bs3 = np.diag(np.ones(num_modes*4))
    
    else:

        bs3_block = np.diag(np.ones(num_modes*2))

        for i in range(num_modes):
            if 2*i+1+(2*bs_3-1) < num_modes*2:
                bs3_block[2*i+1,2*i+1] = 1/np.sqrt(2)
                bs3_block[2*i+1,2*i+1+(2*bs_3-1)] = -1/np.sqrt(2)
                bs3_block[2*i+1+(2*bs_3-1),2*i+1] = 1/np.sqrt(2)
                bs3_block[2*i+1+(2*bs_3-1),2*i+1+(2*bs_3-1)] = 1/np.sqrt(2)

        S_bs3 = np.block([[bs3_block, np.zeros((num_modes*2, num_modes*2))],[np.zeros((num_modes*2, num_modes*2)), bs3_block]]) 

    return S_bs3@S_bs2@S_bs1

def measure_nodes(nodes, Z):
    for node in nodes:
        for i in range(np.shape(Z)[0]):
            Z[node][i] = 0
            Z[i][node] = 0

    return Z

def rotate_node(node, angle, Z):
    
    S_rot = np.diag(np.ones(num_modes*4))
    S_rot[mode][mode] = np.cos(angle)
    S_rot[mode+2*num_modes][mode+2*num_modes] = np.cos(angle)
    S_rot[mode][mode+2*num_modes] = np.sin(angle)
    S_rot[mode+2*num_modes][mode] = -np.sin(angle)

    return  transform_Z(S_rot, Z)

## Plotting Tools

In [3]:
# Coordinates for A & B modes aligned in time

def get_coordinate(i):
    is_odd = i % 2
    i = math.floor(i/2)
    y = math.floor(i/bs_3) + 0.6*is_odd
    i = i % bs_3
    z = 0.5*(math.floor(i/(bs_2)))
    x = (i % (bs_2))
    return [x,y,z]

# Coordinates for A & B modes skewed in time (B modes shifted by one time delay)

def get_coordinate_skewed(i):
    is_odd = i % 2
    i = math.floor(i/2)
    y = math.floor(i/bs_3) + 0.5*is_odd
    i = i % bs_3
    z = math.floor(i/(bs_2))
    x = i % (bs_2)+1*is_odd
    return [x,y,z]

def get_coordinate_skewed_2(i):
    is_odd = i % 2
    i = math.floor(i/2)
    y = math.floor(i/bs_3) + 0.5*is_odd
    i = i % bs_3
    z = math.floor(i/(bs_2))+1*is_odd
    x = i % (bs_2)+1*is_odd
    return [x,y,z]    

# Draw lines between mode coordinates

def draw_line(i,k,a,colour):
    a_coordinates = coordinates[i]
    b_coordinates = coordinates[k]
    ax.plot3D([a_coordinates[0],b_coordinates[0]],[a_coordinates[1],b_coordinates[1]],[a_coordinates[2],b_coordinates[2]], colour, alpha=a)

## Main Simulation

In [15]:
num_modes = 16
r = 2.5
bs_1, bs_2, bs_3 = 1, 8, 81

Z = np.identity(num_modes*2)*np.exp(-2*r)*1j

Z = transform_Z(epr_symplectic(), Z)

bipartition = identify_bipartition(Z) # Identify modes to be transformed for H-graph to Approximate CV cluster state  (ACVCS) conversion

S_h2c = generate_S_h2c(bipartition) # Symplectic matrix for H-graph -> ACVCS

Z = transform_Z(S_h2c, Z) # Adjacency matrix for ACVCS for EPR state

Z1 = transform_Z(cluster_symplectic(bs_1), Z)

Z2 = transform_Z(cluster_symplectic(bs_1, bs_2+1), Z)

In [16]:
# Create figure
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.title('EPR')

#Remove grid lines and axis
plt.grid(b=None)
plt.axis('off')

#Define nodes
a_coord, b_coord = [], []
coordinates = np.array([get_coordinate(i) for i in range(num_modes*2)])

for i in range(len(coordinates)):
    if i%2 == 0:
        a_coord.append(list(coordinates[i]))
    else:
        b_coord.append(list(coordinates[i]))

a_coord, b_coord = np.array(a_coord), np.array(b_coord)

ax.scatter3D(a_coord[:,0], a_coord[:,1], a_coord[:,2], color='red')
ax.scatter3D(b_coord[:,0], b_coord[:,1], b_coord[:,2], color='blue')
ax.set_xlabel('x')
ax.axes.set_xlim3d(left=0, right=bs_2-1)
ax.axes.set_ylim3d(bottom=0, top=1) 
ax.axes.set_zlim3d(bottom=0, top=0.5) 

cov = Z2 

offset = 0.01

for i in range(num_modes*2):
    for k in range(i+1, num_modes*2):
        if cov[i][k]>(1-offset) and cov[i][k]<(1+offset):
            #weight is tanh(2r) 
            draw_line(i,k,0.5,'purple')

        elif cov[i][k]<-(1-offset) and cov[i][k]>-(1+offset):
            #weight is -tanh(2r)
            draw_line(i,k,0.5,'green')

        elif cov[i][k]>0.5*(1-offset) and cov[i][k]<0.5*(1+offset):
            #weight is 1/2*tanh(2r)
            draw_line(i,k,0.3,'purple')

        elif cov[i][k]<-0.5*(1-offset) and cov[i][k]>-0.5*(1+offset):
            #weight is -1/2*tanh(2r)
            draw_line(i,k,0.3,'green')

        elif cov[i][k]<-(1/np.sqrt(2))*(1-offset) and cov[i][k]>-(1/np.sqrt(2))*(1+offset):
            #weight is -1/sqrt(2)*tanh(2r)
            draw_line(i,k,0.35,'green')

        elif cov[i][k]>(1/np.sqrt(2))*(1-offset) and cov[i][k]<(1/np.sqrt(2))*(1+offset):
            #weight is 1/sqrt(2)*tanh(2r)
            draw_line(i,k,0.35,'purple')

        elif cov[i][k]<-(1/(2*np.sqrt(2)))*(1-offset) and cov[i][k]>-(1/(2*np.sqrt(2)))*(1+offset):
            #weight is -1/2*sqrt(2)*tanh(2r)
            draw_line(i,k,0.25,'green')

        elif cov[i][k]>(1/(2*np.sqrt(2)))*(1-offset) and cov[i][k]<(1/(2*np.sqrt(2)))*(1+offset):
            #weight is 1/2*sqrt(2)*tanh(2r)
            draw_line(i,k,0.25,'purple')

        elif cov[i][k]>(1/4)*(1-offset) and cov[i][k]<(1/4)*(1+offset):
            #weight is 1/4*tanh(2r)
            draw_line(i,k,0.2,'purple')

        elif cov[i][k]<(-1/4)*(1-offset) and cov[i][k]>(-1/4)*(1+offset):
            #weight is -1/4*tanh(2r)
            draw_line(i,k,0.2,'green')

        elif cov[i][k]>(1/(4*np.sqrt(2)))*(1-offset) and cov[i][k]<(1/(4*np.sqrt(2)))*(1+offset):
            #weight is 1/4*sqrt(2)*tanh(2r)
            draw_line(i,k,0.15,'purple')

        elif cov[i][k]<(-1/(4*np.sqrt(2)))*(1-offset) and cov[i][k]>(-1/(4*np.sqrt(2)))*(1+offset):
            #weight is -1/4*sqrt(2)*tanh(2r)
            draw_line(i,k,0.15,'green')

        elif cov[i][k]>(1/8)*(1-offset) and cov[i][k]<(1/8)*(1+offset):
            #weight is 1/8*tanh(2r)
            draw_line(i,k,0.1,'purple')

        elif cov[i][k]<(-1/8)*(1-offset) and cov[i][k]>(-1/8)*(1+offset):
            #weight is -1/8*tanh(2r)
            draw_line(i,k,0.1,'green')

        elif abs(cov[i][k])>0.001:
            if cov[i][k] < 0:
                draw_line(i,k,0.5,'brown')
            else:
                draw_line(i,k,0.5,'black')



In [17]:
display(np.real(Z2))
pd.DataFrame(np.real(Z2)).to_csv("adjacency_real1.csv")



array([[ 0.        , -0.4999546 , -0.70704258, ...,  0.        ,
         0.        ,  0.        ],
       [-0.4999546 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.70704258,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.70704258],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -0.4999546 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.70704258,
        -0.4999546 ,  0.        ]])